In [1]:
import requests
import pandas as pd
import numpy as np
import warnings
import datetime
warnings.filterwarnings('ignore')
# ============上市股票df============
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]

# ============上櫃股票df============
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]

# ============上市股票代號+.TW============
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)

# ============上櫃股票代號+.TWO============
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)

# ============concate全部股票代號============
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)


     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1814 rows x 4 columns]


## 全部策略選中股票

In [5]:
date = '2024-04-03'
path = f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/全個股條件篩選/{date}選股.xlsx'
df = pd.read_excel(path)
df['ID'] = df['ID'].apply(lambda x: str(x))
df = df.set_index('ID')
df['市場別'] = None
for i in df.index:
    df.loc[i, '市場別'] = allstock_info.loc[i, '市場別']
all_df = df.copy()
T5_id = all_df.loc[all_df['T5']].index.values
T5_2_id = all_df.loc[all_df['T5-2']].index.values 
T6_id = all_df.loc[all_df['T6']].index.values
T11_id = all_df.loc[all_df['T11']].index.values
T5_2_id = [i for i in T5_2_id if i not in T5_id]
T6_id = [i for i in T6_id if all([i not in T5_id, i not in T5_2_id])]
T11_id = [i for i in T11_id if all([i not in T5_id, i not in T5_2_id, i not in T6_id])]
template = ['T5', 'T5-2', 'T6', 'T11']
ids = [T5_id, T5_2_id, T6_id, T11_id]
for i, t in enumerate(template):
    text = ''
    for id in ids[i]:
        if df.loc[id, '市場別'] == '上櫃':
            text += f'TPEX:{id}\n'
        elif df.loc[id, '市場別'] == '上市':
            text += f'TWSE:{id}\n'
        # create and save to txt
    with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{date}_{t}.txt', 'w', encoding='utf-8') as f:
        f.write(text)
    print(f'{date}-{t}.txt saved')


2024-03-29-T5.txt saved
2024-03-29-T5-2.txt saved
2024-03-29-T6.txt saved
2024-03-29-T11.txt saved


## 每日新增的股票

In [4]:
# ============讀今日和昨日的股票Excel============
path = r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\全個股條件篩選\\'
new_date = '2024-04-02'
old_date = '2024-04-01'
df_old = pd.read_excel(f'{path}{old_date}選股.xlsx')
df_new = pd.read_excel(f'{path}{new_date}選股.xlsx')
df_old.set_index('ID', inplace=True)
df_new.set_index('ID', inplace=True)
          
print(f'New date:{new_date} | Old date:{old_date}')

# ============今日和昨日股票的ID============
df_same_ID = df_new.loc[df_new.index.isin(df_old.index.values)].index.values
df_old = df_old.loc[df_same_ID]
df_new = df_new.loc[df_same_ID]
old_T5_ID = df_old.loc[df_old['T5']].index.values
old_T5_2_ID = df_old.loc[df_old['T5-2']].index.values
old_T6_ID = df_old.loc[df_old['T6']].index.values
old_T11_ID = df_old.loc[df_old['T11']].index.values
new_T5_ID = df_new.loc[df_new['T5']].index.values
new_T5_2_ID = df_new.loc[df_new['T5-2']].index.values
new_T6_ID = df_new.loc[df_new['T6']].index.values
new_T11_ID = df_new.loc[df_new['T11']].index.values

# ============新增和刪除股票的ID============
T5_add_ID = np.sort([i for i in new_T5_ID if i not in old_T5_ID])
T5_drop_ID = np.sort([i for i in old_T5_ID if i not in new_T5_ID])
T6_add_ID = np.sort([i for i in new_T6_ID if i not in old_T6_ID])
T6_drop_ID = np.sort([i for i in old_T6_ID if i not in new_T6_ID])
T11_add_ID = np.sort([i for i in new_T11_ID if i not in old_T11_ID])
T11_drop_ID = np.sort([i for i in old_T11_ID if i not in new_T11_ID])
T5_2_add_ID = np.sort([i for i in new_T5_2_ID if i not in old_T5_2_ID])
T5_2_drop_ID = np.sort([i for i in old_T5_2_ID if i not in new_T5_2_ID])
T5_drop_goodID = np.sort([i for i in T5_drop_ID if all([df_new.loc[i,'S250rate']>=75, df_new.loc[i, 'S20rate']>85])])
T5_2_drop_goodID = np.sort([i for i in T5_2_drop_ID if all([df_new.loc[i,'ES250rate']>=75, df_new.loc[i, 'ES20rate']>85])])
T5_2_add_ID = [i for i in T5_2_add_ID if i not in T5_add_ID]
T6_add_ID = [i for i in T6_add_ID if all([i not in T5_add_ID, i not in T5_2_add_ID])]
T11_add_ID = [i for i in T11_add_ID if all([i not in T5_add_ID, i not in T5_2_add_ID, i not in T6_add_ID])]
template = ['T5', 'T5-2', 'T6', 'T11']
add = [T5_add_ID, T5_2_add_ID, T6_add_ID, T11_add_ID]
for i, t in enumerate(template):
    text = ''
    for id in add[i]:
        id = str(id)
        if allstock_info.loc[id, '市場別'] == '上櫃':
            text += f'TPEX:{id}\n'
        elif allstock_info.loc[id, '市場別'] == '上市':
            text += f'TWSE:{id}\n'
        # create and save to txt
    with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{new_date}_{t}_add.txt', 'w', encoding='utf-8') as f:
        f.write(text)
    print(f'{new_date}-{t}_add.txt saved')

New date:2024-04-02 | Old date:2024-04-01
2024-04-02-T5_add.txt saved
2024-04-02-T5-2_add.txt saved
2024-04-02-T6_add.txt saved
2024-04-02-T11_add.txt saved


: 

## 股票ID轉tradingview格式

In [25]:
stock_id = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx')
industry = '資產股'
stock_id = stock_id[industry]
stock_id = stock_id.loc[stock_id != 0]
for id in stock_id.values:
    id = str(id)
    if allstock_info.loc[id, '市場別'] == '上櫃':
        text += f'TPEX:{id}\n'
    elif allstock_info.loc[id, '市場別'] == '上市':
        text += f'TWSE:{id}\n'
    # create and save to txt
with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{industry}.txt', 'w', encoding='utf-8') as f:
    f.write(text)
print(f'{industry}.txt saved')

資產股.txt saved
